In [1]:
# Подключение библиотек

import os 
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string

In [2]:
# Путь к файлам с данными
path = 'data'

# Переход в папку с данным путём 
os.chdir(path) 

# Считывание всех файлов с данными для датасета в список
datasets = [pd.read_csv(os.path.basename(file), low_memory=False) for file in os.listdir() if file.startswith('data')]

In [3]:
# Объединение данных в один датасет

df = pd.concat(datasets,ignore_index=True).drop_duplicates().sort_values('Number').reset_index(drop=True)
df = df.drop(columns='Unnamed: 0')

In [4]:
df.head()

,Number,Barcode,Name
0,1,5902062112101,"Уровень Topex алюминиевый, тип 600, 150 см (29..."
1,1,5902062009616,"Плоскогубцы Topex комбинированные, 180 мм (32D..."
2,1,5902062008657,"Набор инструментов Topex 38D225, 41 шт. (38D225)"
3,1,5902062008657,"Набор инструментов Topex 38D225, 41 шт. (38D225)"
4,2,5902062032287,"Киянка Topex резиновая O 63 мм, 680 г (02A355)"


In [5]:
# Стемминг текстовых описаний

stemmer = SnowballStemmer('english')

df['Name'] = df['Name'].apply(lambda x: ''.join([stemmer.stem(y) for y in x]))

In [6]:
df.head()

,Number,Barcode,Name
0,1,5902062112101,"уровень topex алюминиевый, тип 600, 150 см (29..."
1,1,5902062009616,"плоскогубцы topex комбинированные, 180 мм (32d..."
2,1,5902062008657,"набор инструментов topex 38d225, 41 шт. (38d225)"
3,1,5902062008657,"набор инструментов topex 38d225, 41 шт. (38d225)"
4,2,5902062032287,"киянка topex резиновая o 63 мм, 680 г (02a355)"


In [7]:
# Удаление символов пунктуации

df['Name'] = df['Name'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [8]:
df.head()

,Number,Barcode,Name
0,1,5902062112101,уровень topex алюминиевый тип 600 150 см 29c606
1,1,5902062009616,плоскогубцы topex комбинированные 180 мм 32d021
2,1,5902062008657,набор инструментов topex 38d225 41 шт 38d225
3,1,5902062008657,набор инструментов topex 38d225 41 шт 38d225
4,2,5902062032287,киянка topex резиновая o 63 мм 680 г 02a355


Формула для метрики похожести:

$\mathrm{similarity} = \frac{|\bigcap_{i=1}^{k} S_k|}{|\bigcup_{i=1}^{k} S_k|}$,

где $S_1, \dots, S_k$ -- текстовые описания (строки из слов).

In [9]:
# Функция для получения множества слов по текстовому описанию
# Вход: текстовое описание
# Выход: множество слов в текстовом описании

def words(sentence):
    sentence += ' '
    w = []
    start = 0
    end = 0
    while end <= len(sentence) - 2:
        if sentence[end+1] == ' ':
            end = end + 2
            w.append(sentence[start:end-1])
            start = end
        else:
            end = end + 1
    return set(w)

# Функция для рассчёта метрики похожести двух текстовых описаний
# Вход: список текстовых описаний
# Выход: список из множеств слов в текстовых описаниях (слово = подстрока, разделённая пробелами)

def similarity(sentences):
    # Преобразование списка описаний к списку множеств слов
    sentences_trans = [words(sentence) for sentence in sentences]
    # Число элементов в пересечении
    size_of_inter = len(set.intersection(*sentences_trans))
    # Число элементов в объединении
    size_of_union = len(set.union(*sentences_trans))
    # Метрика похожести
    similarity_metric = size_of_inter / size_of_union
    # Вывод значения похожести в процентах
    return float('{:.2f}'.format(similarity_metric*100))

In [10]:
# Новая таблица со сгруппированными текстовыми описаниями для каждого номера карточки

data = df.drop(columns='Barcode').groupby('Number').agg(set)

In [11]:
data.head()

,Name
Number,
1,{уровень topex алюминиевый тип 600 150 см 29c6...
2,{киянка topex резиновая o 63 мм 680 г 02a355}
3,"{перфоратор bosch pt gbh 224 dfr 0611273000, л..."
4,{духовой шкаф bosch hbj558yw0q}
5,{стиральная машина bosch wan 24260 by wan24260...


In [12]:
# Вычисление похожести для каждого набора текстовых описаний

data['Similarity'] = data['Name'].apply(similarity)

In [13]:
data.head()

,Name,Similarity
Number,,
1,{уровень topex алюминиевый тип 600 150 см 29c6...,5.56
2,{киянка topex резиновая o 63 мм 680 г 02a355},100.00
3,"{перфоратор bosch pt gbh 224 dfr 0611273000, л...",14.29
4,{духовой шкаф bosch hbj558yw0q},100.00
5,{стиральная машина bosch wan 24260 by wan24260...,8.33


In [14]:
result = pd.merge(df, data.drop(columns='Name'), on='Number', how='inner')

In [15]:
result.head()

,Number,Barcode,Name,Similarity
0,1,5902062112101,уровень topex алюминиевый тип 600 150 см 29c606,5.56
1,1,5902062009616,плоскогубцы topex комбинированные 180 мм 32d021,5.56
2,1,5902062008657,набор инструментов topex 38d225 41 шт 38d225,5.56
3,1,5902062008657,набор инструментов topex 38d225 41 шт 38d225,5.56
4,2,5902062032287,киянка topex резиновая o 63 мм 680 г 02a355,100.00


In [16]:
result.to_csv('../data/result.csv')